In [1]:
import wrds
import pandas as pd
from pathlib import Path

C:\Users\anne-\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = Path.cwd() / "SAMPLE_10Ks"
file_path.mkdir(parents=True, exist_ok=True)


In [3]:
def create_codes(path):
    names_list = []
    cik_list = []
    year_list = []

    for files in path.glob("*.txt"):
        file = files.read_text().splitlines()
        loop = -1
        for lines in file:
            loop = loop + 1
            line = lines.rfind("COMPANY CONFORMED NAME:")
            if line != -1:
                name = file[loop].replace("COMPANY CONFORMED NAME:", "").strip().rstrip()
                names_list.append(name)
                break

    for files in path.glob("*.txt"):
        file = files.read_text().splitlines()
        year = files.name[:-5:-1]
        year = year[::-1]
        year_list.append(year)
        loop = -1
        for lines in file:
            loop = loop + 1
            line = lines.rfind("CENTRAL INDEX KEY:")
            if line != -1:
                cik = file[loop].replace("CENTRAL INDEX KEY:", "").strip().rstrip()
                cik_list.append(cik)
                break
    
    
    company_info = pd.DataFrame()
    company_info["Companies"] = names_list
    company_info["CIK"] = cik_list
    company_info["Year"] = year_list

    return company_info

In [9]:
company_info = create_codes(file_path)

company_info.drop_duplicates()

year_list = list(company_info["Year"])
cik_list = list(company_info["CIK"])
names_list = list(company_info["Companies"])

In [ ]:
db = wrds.Connection()

In [ ]:
tuple_cik = tuple(cik_list)
cik_params = {"cik_list": tuple_cik}

In [ ]:
CIK_query = db.raw_sql("""SELECT cik, cusip, lpermno, linkdt FROM crsp_a_ccm.ccm_lookup
                 WHERE cik IN %(cik_list)s""", params = cik_params)
CUSIP = pd.DataFrame(CIK_query)

cusip_tuple = tuple(CUSIP["cusip"])
cusip_params = {"cusip_list": cusip_tuple}

permno_tuple = tuple(CUSIP["lpermno"])
permno_params = {"permno_list": permno_tuple}

In [ ]:
db.close

In [ ]:
db = wrds.Connection()

In [ ]:
Returns_Query = db.raw_sql("""SELECT permno,yyyy,cusip,annret,anncap  FROM crsp_a_stock.stkannsecuritydata
                 WHERE permno IN %(permno_list)s""", params = permno_params)
RETURNS = pd.DataFrame(Returns_Query)

In [ ]:
db.close